In [204]:
classcode = \
"""
from qcodes.instrument.visa import VisaInstrument
from qcodes import Parameter

class Weinschel_8320(VisaInstrument):

    def __init__(self, name, address, **kwargs):
        super().__init__(name, address, **kwargs)
    
        self.attenuation = Parameter('attenuation', 
                                      unit='dB',
                                      instrument=self,
                                      set_cmd='ATTN ALL {:02.0f}',
                                      get_cmd='ATTN? 1',
                                      vals=vals.Enum(*np.arange(0, 60.1, 2).tolist()),
                                      get_parser=float)
    def _foo_bar() -> None:
        pass
"""


In [107]:
import parso

In [172]:
def parse_init_function_from_str(code: str, classname) -> parso.python.tree.Function:
    module = parso.parse(classcode)
    classes = tuple(child for child in module.children if isinstance(child, parso.python.tree.Class) and child.name.value == classname)
    assert len(classes) == 1
    myclass = classes[0]
    nodes = tuple(child for child in myclass.children if isinstance(child, parso.python.tree.PythonNode))
    assert len(nodes) == 1
    node = nodes[0]
    init_funcs = tuple(child for child in node.children if isinstance(child, parso.python.tree.Function) and child.name.value == '__init__')
    assert len(init_funcs) == 1
    return init_funcs[0]
    



In [173]:
init_func_tree = parse_init_function_from_str(classcode, "Weinschel_8320")

In [174]:
def extract_statements_from_func_node(parso_func: parso.python.tree.Function):
    function_bodys = tuple(child for child in init_func_tree.children if isinstance(child, parso.python.tree.PythonNode) and child.type == "suite")
    assert len(function_bodys) == 1
    function_body = function_bodys[0]
    statement_lines = tuple(child.children[0] for child in function_body.children if isinstance(child, parso.python.tree.PythonNode) and isinstance(child.children[0],  parso.python.tree.ExprStmt))
    
    return statement_lines

    

In [176]:
stm = extract_statements_from_func_node(init_func_tree)[0]

In [178]:
type(stm)

parso.python.tree.ExprStmt

In [199]:
from typing import Optional, Tuple

In [200]:
def extract_code_without_self_from_statement(stm: parso.python.tree.ExprStmt) -> Optional[Tuple[str, str]]:
    lhs = stm.children[0]
    rhs = stm.get_rhs()
    if len(lhs.children) == 2 and lhs.children[0].value == "self":
        # more robust extraction of arglist
        name = lhs.children[1].children[1].value
        arglist = rhs.children[1].children[1]
        to_remove = []
        for i, arg in enumerate(arglist.children):
            if parse_string_or_node(arg):
                to_remove.append(i)
                to_remove.append(i+1)
        to_remove.sort(reverse=True)
        for j in to_remove:
            arglist.children.pop(j)
        return name, rhs.get_code().strip()
    else:
        return {}
    
    
    

In [210]:
def eval_params_from_code(code: str, classname: str) -> Dict[str, Parameter]:
    init_func_tree = parse_init_function_from_str(classcode, "Weinschel_8320")
    stms = extract_statements_from_func_node(init_func_tree)
    
    param_dict = {}

    for stm in stms:
        name, code = extract_code_without_self_from_statement(stm)
        param_dict[name] = eval(code)
    return param_dict

        


In [211]:
eval_params_from_code(classcode, "Weinschel_8320")

{'attenuation': <qcodes.instrument.parameter.Parameter: attenuation at 2363786406152>}

In [195]:
b["attenuation"].get_code().strip()

"Parameter('attenuation', \n                                      unit='dB',\n                                      set_cmd='ATTN ALL {:02.0f}',\n                                      get_cmd='ATTN? 1',\n                                      vals=vals.Enum(*np.arange(0, 60.1, 2).tolist()),\n                                      get_parser=float)"

In [196]:
c = {}
c["attenuation"] = eval(b["attenuation"].get_code().strip())
